In [1]:
import mlx.core as mx
import mlx.nn as nn

import mlx.optimizers as optim
from mlx.utils import tree_map
from functools import partial
import mlx.core as mx
import mlx.nn as nn

from tqdm import tqdm
import argparse
import random
import csv
import json

In [5]:
class gat_layer(nn.Module):
    def __init__(self, num_nodes: int, dim_embed: int, dim_proj: int, n_att_heads: int): 
        super().__init__()

    def __call__(self, node_proj, adjacency_matrix):
        

In [7]:
class gat_layer(nn.Module):
    def __init__(self, num_nodes: int, dim_embed: int, dim_proj: int, n_att_heads: int, num_layers: int, skip_connections: bool, num_out_layers: int): 
        super().__init__()
        
        total_att_size = dim_proj * n_att_heads;

        gat_layer = gat_layer(num_nodes, dim_embed, dim_proj, n_att_heads)

        self.embed_proj = mx.linear(total_att_size)
        self.gat_layers = mx.Sequential([gat_layer * num_layers])
        self.out_layers = mx.Sequential([mx.Linear(total_att_size, total_att_size)] + [mx.Linear(total_att_size, 7)])

    def __call__(self, node_embeddings, adjacency_matrix):
        assert node_embeddings.shape[1] == self.dim_embed, f'Incorrect node embedding size'

        node_proj = self.embed_proj(node_embeddings);

        for layer in self.gat_layers:
            new_node_emebeddings = layer(node_embeddings)
            if (self.skip_connections):
                new_node_embeddinsg = node_embeddings;
            node_embeddings = node_embeddings

        for layer in self.out_layers:
            node_embeddings = nn.leaky_relu(mx.layer(node_embeddings), .02)

        return mx.softmax(node_embeddings)

5